In [ ]:
import glob, scipy, json, re, time, pickle
import pandas as pd
import workers
from multiprocessing import Pool
import numpy as np
import matplotlib.pyplot as plt
import feather, pytz, spacy, datetime
from itertools import chain
from datetime import timezone
import en_core_web_sm
nlp = en_core_web_sm.load()

from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('bert-base-nli-mean-tokens')

# Preprocess replies

In [2]:
def extract_tweet_info(x):
    return [x['id'], int(x['created_at']/1000), x.get('user_id', ''), x['tweet']]

file_list = glob.glob('../dat/hks_misinformation_review/covid_antibiotics_replies/*.json')

sampled_tweets = []
for file in file_list:
    for line in open(file, 'r', encoding='utf-8'):
        sampled_tweets.append(extract_tweet_info(json.loads(line)))
        
sampled_tweets_df = pd.DataFrame(sampled_tweets, columns=['id', 'created', 'uid', 'text'])
print(sampled_tweets_df.shape)

(16396, 4)


In [5]:
covid_df = sampled_tweets_df.copy()
covid_df.index = covid_df.created.apply(lambda x: datetime.datetime.fromtimestamp(x, pytz.utc))
covid_df = covid_df.sort_index()

criteria  = (covid_df.index.year==2020)
covid_df = covid_df[criteria]
print("size of df in the year 2020:", covid_df.shape)

covid_df = covid_df.drop_duplicates()
print("size of df after dropping duplicates:", covid_df.shape)

size of df in the year 2020: (16383, 4)
size of df after dropping duplicates: (16383, 4)
